# Obteniendo datos de la API de Spotify

In [1]:
import base64
import requests
import pandas as pd
import os
import discogs_client
import time

## Autorización

In [2]:
client_id = os.environ['spotify_client_id']
client_secret = os.environ['spotify_client_secret']

In [3]:
# función de autenticación de spotify

def spoty_aut (client_id, client_secret):
    url = 'https://accounts.spotify.com/api/token'
    credenciales = client_id + ':' + client_secret
    authorization = "Basic " + base64.b64encode(credenciales.encode()).decode()
    grant_type= 'client_credentials'
    auth = requests.post(url, data={'grant_type':grant_type},
              headers={'Authorization':authorization, 'Content-Type':'application/x-www-form-urlencoded'})
    return auth.json()

In [4]:
auth=spoty_aut(client_id, client_secret)

In [5]:
auth

{'access_token': 'BQAAO3pzZwKqlwsjuzQEK3UAOj7k1a50HdSThtBPT6weFHtpwZezra9q5JIdaUC94RpiCkaTEMBdQYXPfLc',
 'token_type': 'Bearer',
 'expires_in': 3600}

In [6]:
# autorización de la API de discogs
d = discogs_client.Client('ExampleApplication/0.1', user_token="TetTWkQitrxfTKxUfBUaGidepABvNfmADZwDYRfm")

## Obteniendo playlist

In [7]:
playlist_id="5xzPwmpI3aCPfNk4YnuQyG"

In [8]:
#offset
base = "https://api.spotify.com/v1/"
playlist = requests.get(base+f"playlists/{playlist_id}/tracks?offset=0",
                    headers = {'Authorization':f'''Bearer {auth['access_token']}''', 'Content-Type': 'application/json'})
    

In [198]:
#playlist.json()['next']==None

True

In [23]:
#función que crea df con variables necesarias para sherpa a partir de playlist_id

def df_sherpa(playlist_id):
    base = "https://api.spotify.com/v1/"
    playlist = requests.get(base+f"playlists/{playlist_id}/tracks?offset=200",
                      headers = {'Authorization':f'''Bearer {auth['access_token']}''', 'Content-Type': 'application/json'})
    
    n_canciones=len(playlist.json()['items'])
    
    lista_df_tracks=[]
    
    for i in range(0,n_canciones):
        
        #obteniendo autor, nombre de canción y sus IDs de las canciones en la playlist
        artist_track=playlist.json()['items'][i]
        
        artist_id= artist_track['track']['artists'][0]['id']
        artist = artist_track['track']['artists'][0]['name']
        track = artist_track['track']['name']
        track_id = artist_track['track']['id']
        album = artist_track['track']['album']['name']
        duration=artist_track['track']['duration_ms']/60_000
        
        #obteniendo características de canciones de playlist
        audio_features = requests.get(base+f"audio-features/{track_id}",
                          headers = {'Authorization':f'''Bearer {auth['access_token']}''', 'Content-Type': 'application/json'})
        tempo = audio_features.json()['tempo']
        energy = audio_features.json()['energy']
        danceability = audio_features.json()['danceability']
        valence = audio_features.json()['valence']
        key = audio_features.json()['key']
        time_signature = audio_features.json()['time_signature']
        acousticness = audio_features.json()['acousticness']
        instrumentalness = audio_features.json()['instrumentalness']
        mode = audio_features.json()['mode']
    
        lista_track = [track_id, artist, track, album, duration, tempo, energy, danceability, 
                 valence, key, time_signature, acousticness, instrumentalness, mode]
        lista_df_tracks.append(lista_track)
    
    df_sherpa=pd.DataFrame(lista_df_tracks, columns=["track_id", "artist", "track", "album", "duration",  "tempo", "energy", "danceability", 
             "valence", "key", "time_signature", "acousticness", "instrumentalness", "mode"])
    return df_sherpa
    
    

In [24]:
#Aplicando la función para obtener variables de estudio de canciones de una playlist
df_test=df_sherpa(playlist_id)

In [15]:
#df_test_full=df_test

In [25]:
df_test_full = pd.concat([df_test_full, df_test])

In [37]:
df_test_full.to_csv("/Users/santiago/ih-bootcamp/proyectos/sherpa/out/ambient_set.csv")

In [38]:
df_test = pd.read_csv("/Users/santiago/ih-bootcamp/proyectos/sherpa/out/ambient_set.csv")

## Discogs database

In [39]:

styles=[]

for track_id in df_test['track_id']:
    base = "https://api.spotify.com/v1/"
    song = requests.get(base+f"tracks/{track_id}",
                      headers = {'Authorization':f'''Bearer {auth['access_token']}''', 'Content-Type': 'application/json'})
    
    album=song.json()['album']['name']
    
     
    results = d.search(f'{album}')
    if len(results)==0:
        styles.append(['Not data', album])
    else:    
        if str(results[0]).startswith('<Label') :
            if str(results[1]).startswith('<Release') :
                release=results[1].id
                styles.append([d.release(release).styles, album])
            elif str(results[1]).startswith('<Master'):
                release=results[1].id
                styles.append([d.master(release).styles, album])
        else:
            if str(results[0]).startswith('<Release') :
                release=results[0].id
                styles.append([d.release(release).styles, album])
            elif str(results[0]).startswith('<Master'):
                release=results[0].id
                styles.append([d.master(release).styles, album])    

In [40]:
styles=pd.DataFrame(styles, columns=['styles', 'album'])

In [41]:
df_test=pd.merge(df_test, styles, on='album', how='left')

In [42]:
dic={0:"C", 1:"C#", 2:"D", 3:"D#", 4:"E", 5:"F", 6:"F#", 7:"G", 8:"G#", 9:"A", 10:"A#", 11:"B"}
df_test=df_test.replace({"key":dic})

In [43]:
df_test=df_test.loc[df_test.astype(str).drop_duplicates().index].reset_index()

In [44]:
df_test.to_csv("/Users/santiago/ih-bootcamp/proyectos/sherpa/out/ambient_set.csv")